# Desc

- using this file to practice/learn basic stats
- also want to learn standard time series analysis techniques

In [1]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.ar_model import AutoReg

from sklearn.metrics import r2_score

In [4]:

def check_stationary(series):
    # Copied from https://machinelearningmastery.com/time-series-data-stationary-python/

    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.01) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

In [5]:
# parameters
_round_num = 4
_time_per_day = 1000000

In [6]:
def get_file_trades(day):
    fname = f"../data/round5/trades_round_{_round_num}_day_{day}_wn.csv"
    print("fname", fname)
    return pd.read_csv(fname, sep=";")


def get_file_prices(day):
    fname = f"../data/round{_round_num}/prices_round_{_round_num}_day_{day}.csv"
    print("fname", fname)
    return pd.read_csv(fname, sep=";")

def get_symbol_prices(symbol, days):
    """ Returns DataFrame containing all price data for specified symbol, over the specified 'days'
    If symbol is None, don't filter on symbol
    """
    
    data = []
    for day in days:
        price_df = get_file_prices(day)
        data += [price_df]
    
    # concat
    price_df = pd.concat(data)
    
    # rename "bid" to "buy"
    # rename "ask" to "sell"
    price_df = price_df.rename({col: col.replace("bid", "buy") for col in price_df.columns if "bid" in col}, axis=1)
    price_df = price_df.rename({col: col.replace("ask", "sell") for col in price_df.columns if "ask" in col}, axis=1)
    price_df = price_df.rename({"timestamp": "time", "product": "symbol"}, axis=1)
    
    if symbol is not None:
        # filter for specific symbol
        price_df = price_df[price_df["symbol"] == symbol]
    
    # fix times, sort by time and reindex
    price_df["time"] = price_df["time"] + (price_df["day"] - min(days)) * _time_per_day
    price_df = price_df.sort_values(by="time").reset_index(drop=True)
    
    return price_df

def get_symbol_trades(symbol, days):
    data = []
    for day in days:
        price_df = get_file_trades(day)
        price_df["day"] = day
        data += [price_df]
    
    # concat
    price_df = pd.concat(data)
    price_df = price_df.rename({"timestamp": "time", "product": "symbol"}, axis=1)
    
    if symbol is not None:
        # filter for specific symbol
        price_df = price_df[price_df["symbol"] == symbol]
    
    # fix times, sort by time and reindex
    price_df["time"] = price_df["time"] + (price_df["day"] - min(days)) * _time_per_day
    price_df = price_df.sort_values(by="time").reset_index(drop=True)
    
    return price_df
    


def get_all_mids(days):
    price_df = get_symbol_prices(symbol=None, days=days)
    
    symbols = set(price_df["symbol"])
    
    price_df["mid"] = (price_df["buy_price_1"] + price_df["sell_price_1"]) / 2
    
    return price_df[["time", "symbol", "mid"]].copy()

In [7]:
def get_book_large_size(price_df, side):
    """ 
    side = 'buy' or 'sell' 
    
    Returns prices, sizes
    """
    
    book_range = list(range(1, 3+1))
    
    # get bids
    bid_sizes = price_df[[f"{side}_volume_{i}" for i in book_range]]
    bid_prices = price_df[[f"{side}_price_{i}" for i in book_range]]

    # replace nan with -1
    bid_sizes = bid_sizes.replace(np.nan, -1)

    # find inds
    inds = np.argmax(bid_sizes.to_numpy(), axis=1)

    # filter to keep largest bids
    bid_prices = bid_prices.to_numpy()[np.arange(len(bid_prices)), inds]
    bid_sizes = bid_sizes.to_numpy()[np.arange(len(bid_sizes)), inds]
    
    # create dataframe
    df = pd.DataFrame([bid_prices, bid_sizes]).T
    df.columns = [f"{side}_price", f"{side}_size"]
    df["time"] = price_df["time"].reset_index()["time"]
    
    return df


def get_whale_quotes(price_df):
    # get big buy/sell quotes
    buy_df = get_book_large_size(price_df, "buy")
    sell_df = get_book_large_size(price_df, "sell")

    # create big quote_df
    quote_df = pd.merge(buy_df, sell_df, on="time")
    quote_df["mid_price"] = (quote_df["buy_price"] + quote_df["sell_price"]) / 2
    quote_df["spread"] = quote_df["sell_price"] - quote_df["buy_price"]

    return quote_df

# Counterparty analysis

In [8]:
# load data
# _day_range = [-2, -1, 0]
# _day_range = [-1, -0, 1]
# _day_range = [0, 1, 2]
_day_range = [1, 2, 3]
trade_df = get_symbol_trades(None, days=_day_range)
price_df = get_symbol_prices(None, days=_day_range)
_symbols = set(trade_df["symbol"])


# compute all_mid_Df
all_dfs = []

for sym in _symbols:
    df = price_df[price_df["symbol"] == sym]
    
    datas = {
        "time": df["time"],
        "mid": df["mid_price"],
        "symbol": df["symbol"]
    }

    for i in [1, 3, 10, 30, 100, 300, 1000, 3000, 5000]:
        datas[f"d_{i}"] = -1 * df["mid_price"].diff(-1 * i)

    all_dfs += [pd.DataFrame(datas)]

all_mid_df = pd.concat(all_dfs)

# merge into trade_df
trade_df = trade_df.merge(all_mid_df, on=["symbol", "time"])
trade_df["sell_edge"] = trade_df["price"] - trade_df["mid"]

fname ../data/round5/trades_round_4_day_1_wn.csv
fname ../data/round5/trades_round_4_day_2_wn.csv
fname ../data/round5/trades_round_4_day_3_wn.csv
fname ../data/round4/prices_round_4_day_1.csv
fname ../data/round4/prices_round_4_day_2.csv
fname ../data/round4/prices_round_4_day_3.csv


In [13]:
all_market_trades = {}
for index, row in trade_df.iterrows():
    time = row["time"]
    sym = row["symbol"]
    if time not in all_market_trades:
        all_market_trades[time] = {}
    if sym not in all_market_trades[time]:
        all_market_trades[time][sym] = []
    
    trade = Trade(
        symbol=row["symbol"],
        price=row["price"], 
        quantity=row["quantity"], 
        buyer=row["buyer"], 
        seller=row["seller"],
    )
        
    all_market_trades[time][sym] += [trade]
    

NameError: name 'Trade' is not defined

In [11]:
all_market_trades[100]

{'PEARLS': [{'time': 100,
   'buyer': 'Paris',
   'seller': 'Charlie',
   'symbol': 'PEARLS',
   'currency': 'SEASHELLS',
   'price': 10002.0,
   'quantity': 1,
   'day': 1,
   'mid': 10003.0,
   'd_1': -3.0,
   'd_3': -2.0,
   'd_10': -4.0,
   'd_30': -2.0,
   'd_100': -0.0,
   'd_300': -3.0,
   'd_1000': -6.0,
   'd_3000': -3.0,
   'd_5000': -1.0,
   'sell_edge': -1.0}],
 'BANANAS': [{'time': 100,
   'buyer': 'Paris',
   'seller': 'Charlie',
   'symbol': 'BANANAS',
   'currency': 'SEASHELLS',
   'price': 4875.0,
   'quantity': 1,
   'day': 1,
   'mid': 4876.0,
   'd_1': -3.0,
   'd_3': -5.5,
   'd_10': -5.5,
   'd_30': -3.5,
   'd_100': -6.5,
   'd_300': -6.5,
   'd_1000': 5.5,
   'd_3000': -31.0,
   'd_5000': -53.5,
   'sell_edge': -1.0}]}

In [ ]:
all_mid_df['symbol'].unique()

In [ ]:
# general plot of data
total_buys = trade_df.groupby(["buyer"])[["quantity"]].sum()
total_sells = trade_df.groupby(["seller"])[["quantity"]].sum()

plt.figure(figsize=(10, 5))

plt.subplot(121)
plt.bar(total_buys.index, total_buys["quantity"])
plt.xticks(rotation=45)
plt.subplot(122)
plt.bar(total_sells.index, total_sells["quantity"])
plt.xticks(rotation=45)
plt.show()

In [ ]:
df1 = trade_df.groupby(["symbol", "buyer"])["quantity"].agg(["sum", "count"])
df2 = trade_df.groupby(["symbol", "seller"])["quantity"].agg(["sum", "count"])

df1 = df1.reset_index().rename({"buyer": "party", "sum": "buy_vol", "count": "buy_count"}, axis=1)
df2 = df2.reset_index().rename({"seller": "party", "sum": "sell_vol", "count": "sell_count"}, axis=1)

vol_df = df1.merge(df2, on=["symbol", "party"], how="outer")

for sym in _symbols:
    df = vol_df[vol_df["symbol"] == sym].sort_values(by="buy_vol", ascending=False)
#     df = df[(df["buy_count"] < 20)]
#     display(df.drop(["buy_count", "sell_count"], axis=1))
    display(df)

In [ ]:
party = "Olivia"

party_trades = trade_df[(trade_df["buyer"] == party) | (trade_df["seller"] == party)]
print(len(party_trades))

syms = set(party_trades["symbol"])

for sym in syms:
    plt.figure(figsize=(13, 5))
    df = price_df[price_df["symbol"] == sym].reset_index()
    
    tdf = party_trades[party_trades["symbol"] == sym]
    

    for index, row in tdf.iterrows():
        is_buy = row["buyer"] == party
#         if not is_buy:
#             continue
        color = "green" if is_buy else "red"
        time = row["time"] // 100
        plt.axvline(x=time, color=color)
    
    plt.plot(df["mid_price"])

    plt.title(sym)
    plt.show()

In [ ]:
trade_df

In [ ]:
len(trade)d

## Analysis per symbol per trader

In [ ]:
trade_df['symbol'].unique()

In [ ]:
#SYMBOL TO USE FOR NEXT FOUR CELLS
symbol = "BAGUETTE"

# CELL INFO
# For this symbol, this cell gets data on trades for each trader
#'time' - timestep 
#'position' - position 
#'money' - money 
#'n_buy' - number of total buy trades 
#'n_sell' - number of total sell trades 
#'n_trade' - number of total trades 
#'is_buy' - whether it was a buy (true if buy)

trade_df = trade_df.sort_values(by='time')
trade_df_symbol = trade_df[(trade_df['symbol'] == symbol) | (trade_df['symbol'] == symbol)] 
names_list = pd.concat([trade_df['buyer'], trade_df['seller']]).unique()


position_df_dict = {name : pd.DataFrame(columns=['time', 'position', 'money', 'n_buy', 'n_sell', 'n_trade', 'is_buy']) for name in names_list}
position_dict = {name : 0 for name in names_list}
money_dict = {name : 0 for name in names_list}
buy_count_dict = {name : 0 for name in names_list}
sell_count_dict = {name : 0 for name in names_list}

for index, row in trade_df_symbol.iterrows():
    buyer = row['buyer']
    seller = row['seller']
    price = row['price']
    quantity = row['quantity']
    time = row['time']
    
    #update buyer position
    position_dict[buyer] += quantity
    money_dict[buyer] -= (quantity * price)
    buy_count_dict[buyer] += 1
    new_row = {'time': time, 
               'position': position_dict[buyer], 
               'money': money_dict[buyer],
               'n_buy': buy_count_dict[buyer],
               'n_sell': sell_count_dict[buyer],
               'n_trade': buy_count_dict[buyer]+sell_count_dict[buyer],
               'is_buy': True,
              }
    position_df_dict[buyer].loc[len(position_df_dict[buyer])] = new_row
    
    #update seller position
    position_dict[seller] -= quantity
    money_dict[seller] += (quantity * price)
    sell_count_dict[seller] += 1
    new_row = {'time': time, 
               'position': position_dict[seller], 
               'money': money_dict[seller],
               'n_buy': buy_count_dict[seller],
               'n_sell': sell_count_dict[seller],
               'n_trade': buy_count_dict[seller]+sell_count_dict[seller],
               'is_buy': False,
              }
    
    position_df_dict[seller].loc[len(position_df_dict[seller])] = new_row
    

In [ ]:
# CELL INFO
# Get final prices for each of the symbols
symbol_list = all_mid_df['symbol'].unique()
all_mid_df_group_symbol = all_mid_df.groupby(["symbol"])


final_price_dict = {}

for sym in symbol_list:
    all_mid_df_sym = all_mid_df_group_symbol.get_group(sym)
    last_mid_price = all_mid_df_sym['mid'].iloc[-1]
    final_price_dict[sym] = last_mid_price
    
#get final price for symbol we are looking at
final_price_sym = final_price_dict[symbol]

In [ ]:
#CELL INFO
#get final PnL for each trader for the symbol
final_pnl_dict = {}
for name in position_df_dict:
    #check if made no trades
    if len(position_df_dict[name]) == 0:
        final_pnl_dict[name] = 0
    else:
        final_money = position_df_dict[name]['money'].iloc[-1]
        final_position = position_df_dict[name]['position'].iloc[-1]
        final_pnl_dict[name] = final_money + final_position * final_price_sym

In [ ]:
#CELL INFO
#plot positions for each trader
symbol_price_df = all_mid_df_group_symbol.get_group(symbol)



time_range = (0, float('inf'))
#filter by time
symbol_price_df = symbol_price_df[(time_range[0] <= symbol_price_df['time']) & (symbol_price_df['time'] <= time_range[1])]


fig, axs = plt.subplots(len(position_df_dict), 2, figsize=(10, 3*len(position_df_dict)))

for i, (name, df) in enumerate(position_df_dict.items()):
    #filter by time
    df = df[(time_range[0] <= df['time']) & (df['time'] <= time_range[1])]

    
    # plot the time vs. position
    axs[i][0].plot(df['time'], df['position'])
    axs[i][0].set_title(name)
    
    
    #plot the symbol price data and vertical lines where the trader bought/sold
    axs[i][1].plot(symbol_price_df['time'], symbol_price_df['mid'])
    # loop through the rows of df
    if len(df) == 0:
        continue
    for index, row in df.iterrows():
        # check if is_buy is True
        if row['is_buy']:
            # plot a green vertical line at the time value where buy
            axs[i][1].axvline(x=row['time'], color='green')
        else:
            # plot a red vertical line at the time value where sell
            axs[i][1].axvline(x=row['time'], color='red')  
    
    axs[i][1].set_title(f'Final PnL for {name} on {symbol}: {final_pnl_dict[name]}')

plt.tight_layout()
plt.show()

# General Analysis Strategy
- find who is making the most money
- find their most profitable types of trades
    - filter by symbols + buy/sell side
    - map their return after 1, 3, 10, 30, ... turns

In [ ]:
cols = ["count", "sell_edge"] + [col for col in trade_df.columns if col.startswith("d_")]
sort_by = "d_100"

# in buyers, most positive is best
counts = trade_df.groupby(["buyer", "symbol"])["time"].count()
buys = trade_df.groupby(["buyer", "symbol"]).mean()
buys["count"] = counts
display(buys[cols].sort_values(by=sort_by, ascending=False))
# display(buys[cols])

# in sellers, most negative is best
counts = trade_df.groupby(["seller", "symbol"])["time"].count()
sells = trade_df.groupby(["seller", "symbol"]).mean()
sells["count"] = counts
display(sells[cols].sort_values(by=sort_by, ascending=True))


## Pairs trading

In [ ]:
_day_range = [1, 2, 3]
# _day_range = [-1, 0, 1]

coco_price_df = get_symbol_prices("COCONUTS", _day_range)
pina_price_df = get_symbol_prices("PINA_COLADAS", _day_range)

coco_df = get_whale_quotes(coco_price_df)
pina_df = get_whale_quotes(pina_price_df)
pair_df = pd.DataFrame({
    "mid_coco": coco_df["mid_price"], 
    "mid_pina": pina_df["mid_price"],
    "time": pina_df["time"],
})

fig, ax = plt.subplots()

ax.plot(pair_df["time"], pair_df["mid_coco"], color="blue", label="coco")
ax.legend(loc=1)

ax2 = ax.twinx()

ax2.plot(pair_df["time"], pair_df["mid_pina"], color="orange", label="pina")
ax2.legend(loc=2)

### plot first diffs over time

In [ ]:
plt.scatter(pair_df["mid_coco"].diff(1), pair_df["mid_pina"].diff(1))

In [ ]:
plt.figure(figsize=(12, 3))

all_x = pair_df["mid_coco"].diff(1).dropna()
all_y = pair_df["mid_pina"].diff(1).dropna()

xlim = all_x.min(), all_x.max()
ylim = all_y.min(), all_y.max()

big_model = np.poly1d(np.polyfit(all_x, all_y, deg=1))
print("big_model", big_model.coef)

### params
chunk_size = 100


model_data = []

for i in range(len(pair_df) - chunk_size):

    lb = i
    ub = lb + chunk_size

    df = pair_df.iloc[lb:ub]

    x = df["mid_coco"].diff(1).dropna()
    y = df["mid_pina"].diff(1).dropna()

    inds = x.index.intersection(y.index)

    x = x[inds]
    y = y[inds]

    # fit line
    model = np.poly1d(np.polyfit(x, y, deg=1))
    model_data += [{
        "m": model.coef[0],
        "b": model.coef[1],
        "r2": r2_score(y, model(x)),
    }]

#     if i < 5:
#         plt.subplot(121)
#         plt.hist(x, bins=50)
#         plt.hist(y, bins=50, alpha=0.5)

#         plt.subplot(122)
#         plt.scatter(x, y, s=1)
#         plt.plot(x, model(x), color="red", label="fit")
#         plt.plot(x, model(x), color="black", label="fit")
#         plt.xlim(xlim)
#         plt.ylim(ylim)
        
#         plt.show()

#         print("model", model.coef)
model_df = pd.DataFrame(model_data)

In [ ]:
plt.plot(model_df["r2"], label="r2")
plt.plot(model_df["m"], label="m")
plt.plot(model_df["b"], label="b")
plt.axhline(y=1.5)

plt.legend()


In [ ]:
check_stationary(model_df["m"])

In [ ]:
# get errors
error = pair_df["mid_pina"] - pair_df["mid_coco"] * 1.5 - 3000
error_df = pd.DataFrame({"error": error})

# check stationarity of the error
check_stationary(error_df["error"])

# get `change` stats
error_df["above"] = error_df["error"] > 0
error_df["change"] = error_df["above"] != error_df["above"].shift(1)


In [ ]:
plt.plot(error_df["error"])

In [ ]:
error.describe()

In [ ]:
low, high = 0, 0

bounds = []

for index, row in error_df.iterrows():
    if row["change"]:
        bounds += [(low, high)]
        low, high = 0, 0
        
    # record max error
    low = min(low, row["error"])
    high = max(low, row["error"])
    
    
df = pd.DataFrame(bounds, columns=["low", "high"])
df["bound"] = np.maximum(df["low"].abs(), df["high"].abs())

# get number of times we reach each threshold
freq_df = df.groupby("bound")[["low"]].count().rename({"low": "count"}, axis=1)
freq_df = freq_df.sort_index(ascending=False).cumsum().reset_index()
freq_df["pnl"] = freq_df["bound"] * freq_df["count"]

In [ ]:
plt.plot(freq_df["bound"], freq_df["pnl"])

In [ ]:
freq_df.sort_values(by="pnl", ascending=False).iloc[:20]

## round 4

In [ ]:
del pair_df

In [ ]:
_day_range = [1, 2, 3]

raw_df = get_symbol_prices(None, _day_range)

syms = ["DIP", "BAGUETTE", "UKULELE", "PICNIC_BASKET"]

all_dfs = {}
for sym in syms:
    df = raw_df[raw_df["symbol"] == sym]
    df = get_whale_quotes(df)
    all_dfs[sym[:3].lower()] = df["mid_price"]
#     all_dfs["time"] = df["time"]
    

price_df = pd.DataFrame(all_dfs)

In [ ]:
price_df["agg"] = 2 * price_df["bag"] + 4 * price_df["dip"] + 1 * price_df["uku"]
price_df["orig_error"] = price_df["pic"] - price_df["agg"]

In [ ]:
check_stationary(price_df["orig_error"])
display(price_df["orig_error"].describe().to_frame().T)

In [ ]:
plt.plot(price_df["orig_error"])
plt.show()
plt.hist(price_df["orig_error"])
plt.show()

In [ ]:
def get_model(weights):
    return lambda x : np.dot(weights, list(x) + [1])

model = get_model([1, -4, -2, -1, -375])



In [ ]:
price_df["bias"] = 1
price_df["error"] = price_df[["pic", "dip", "bag", "uku"]].apply(model, axis=1)


In [ ]:
error_df = price_df[["error"]].copy()

# check stationarity of the error
check_stationary(error_df["error"])

# get `change` stats
error_df["above"] = error_df["error"] > 0
error_df["change"] = error_df["above"] != error_df["above"].shift(1)

In [ ]:
## add fees here

spreads = {
    "pic": 11.377,
    "uku": 1.419,
    "dip": 1.674,
    "bag": 1.596,
}

spread_fee = np.sum(list(spreads.values())) / 2
print("spread fee", spread_fee)

##

low, high = 0, 0

bounds = []

for index, row in error_df.iterrows():
    if row["change"]:
        bounds += [(low, high)]
        low, high = 0, 0
        
    # record max error
    low = min(low, row["error"])
    high = max(low, row["error"])
    
    
df = pd.DataFrame(bounds, columns=["low", "high"])
df["bound"] = np.maximum(df["low"].abs(), df["high"].abs())

# get number of times we reach each threshold
freq_df = df.groupby("bound")[["low"]].count().rename({"low": "count"}, axis=1)
freq_df = freq_df.sort_index(ascending=False).cumsum().reset_index()

### adjust profit by spread, you have to pay it twice
freq_df["pnl"] = (freq_df["bound"] - 2 * spread_fee) * freq_df["count"]

In [ ]:
plt.plot(freq_df["bound"], freq_df["pnl"])

In [ ]:
freq_df.sort_values(by="pnl", ascending=False).iloc[:20]